In [73]:
import numpy as np
from collections import OrderedDict as odict
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

This notebook computes the latencies and bandwidths of the three primitive function types

In [74]:
#(hardware name, number of nodes)
files = odict({})
files['i5'] = ('i5',1)
files['gtx1060'] = ('gtx1060',1)
files['skl_mpi1'] = ('skl',1)
files['skl_mpi2'] = ('skl',2)
files['skl_mpi4'] = ('skl',4)
files['knl_mpi1'] = ('knl',1)
files['knl_mpi2'] = ('knl',2)
files['knl_mpi4'] = ('knl',4)
files['p100_mpi1'] = ('p100',1)
files['p100_mpi2'] = ('p100',2)
files['p100_mpi4'] = ('p100',4)
files['v100_mpi1'] = ('v100',1)
files['v100_mpi2'] = ('v100',2)
files['v100_mpi4'] = ('v100',4)

pd.set_option('precision',2)

#### Axpby and Dot Latencies
The latencies are determined by taking the minimum of the average runtimes 

#### Axpby and Dot Bandwidths
The bandwidths are determined by taking the average bandwidth of the 30 bandwidths corresponding to the 3 largest sizes.

#### Dx-Dy Latencies
As in Axpby 

#### Dx-Dy Bandwidths
Since the efficiency of the matrix-vector multiplications depends on the polynomial coefficient we should compute these bandwidths separately


In [87]:
names={'axpby':3,'dot':2,'dx':3, 'dy':3}
#ns=[3,4]
values = []
for f, v in files.items() :
    runtimes=pd.read_csv('benchmark_'+f+'.csv', delimiter=' ')
    #add size and bandwidth columns
    runtimes.insert(0,'size', 8*runtimes['n']*runtimes['n']
                    *runtimes['Nx']*runtimes['Ny']/1e6/v[1]) #inplace transformation
    for name,memops in names.items() :
        runtimes.insert(0,name+'_bw',runtimes['size']/1000*memops/runtimes[name])
    runtimes = runtimes.assign( dxdy=(runtimes['dx']+runtimes['dy'])/2)
    runtimes = runtimes.assign( dxdy_bw=2.0*runtimes['dx_bw']*runtimes['dy_bw']
                               /(runtimes['dx_bw']+runtimes['dy_bw']))
    #compute one version with aggregated grouped sizes and one without
    avgruntimes=runtimes.groupby(['n', 'Nx','Ny','size']).agg(['mean', 'std'])
    avgruntimes=avgruntimes.reset_index(level=['n','Nx','Ny','size'])
    avgruntimes.sort_values(by='size',inplace=True) #sort by size
    runtimes.sort_values(by='size',inplace=True)
    ##first compute axpby and dot latencies and bandwidths 
    nmax = 3
    s =30

    line = []
    l=len(runtimes)
    line.append(v[0]) #0
    line.append(v[1]) #1
    for q in ['axpby','dot']:
        bandwidth       = runtimes[l-s:l][q+'_bw'].mean()
        err_bandwidth   = runtimes[l-s:l][q+'_bw'].std()
        mean_latency    = avgruntimes[0:nmax][(q,'mean')].mean()/1e-6
        min_latency     = avgruntimes[(q,'mean')].min()/1e-6
        idx_min_latency = avgruntimes[(q,'mean')].idxmin()
        corr_min_latency= min_latency - avgruntimes['size'].loc[idx_min_latency]*names[q]/bandwidth/1e-3
        err_latency     = avgruntimes[(q,'std')].loc[idx_min_latency]/1e-6
        if corr_min_latency <0 : corr_min_latency = 0 
        line.append(bandwidth) #2 bandwidth
        line.append(err_bandwidth)  #3 err_bandwidth
        line.append(mean_latency) #4 latency mean
        line.append(min_latency)  #5 latency min
        line.append(corr_min_latency ) #6 corrected lat 
        line.append(err_latency ) #6 error lat 
    ##now compute latency and bandwidths of dx and y
    for n in [2,3,4,5]:
        #take n
        dxdy=runtimes[runtimes['n']==n]
        
        avgdxdy = avgruntimes[avgruntimes['n']==n]
        dxdy=dxdy.sort_values(by='size')
        avgdxdy=avgdxdy.sort_values(by='size') #sort by size
        bandwidth       = dxdy[(dxdy['size']>10)&(dxdy['size']<1000)]['dxdy_bw'].mean()
        err_bandwidth   = dxdy[(dxdy['size']>10)&(dxdy['size']<1000)]['dxdy_bw'].std()
        mean_latency    = avgdxdy[0:nmax][('dxdy','mean')].mean()/1e-6
        min_latency     = avgdxdy[('dxdy','mean')].min()/1e-6
        idx_min_latency = avgdxdy[('dxdy','mean')].idxmin()
        corr_min_latency= min_latency - avgdxdy['size'].loc[idx_min_latency]*names['dx']/bandwidth/1e-3
        err_latency     = avgdxdy[('dxdy','std')].loc[idx_min_latency]/1e-6
        if corr_min_latency <0 : 
            corr_min_latency = 0 
        line.append(bandwidth) #2 bandwidth
        line.append(err_bandwidth)  #3 err_bandwidth
        line.append(mean_latency) #4 latency mean
        line.append(min_latency)  #5 latency min
        line.append(corr_min_latency ) #6 corrected lat  
        line.append(err_latency ) #6 error lat     
    values.append(line)

In [86]:
#now construct new table with values from previous cell      
tuples=[('arch','',''),('nodes','','')]        
for q in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append((q,'bw','avg'))
    tuples.append((q,'bw','std'))
    tuples.append((q,'lat','avg'))
    tuples.append((q,'lat','min'))
    tuples.append((q,'lat','corr'))
    tuples.append((q,'lat','std'))
    
cols=pd.MultiIndex.from_tuples(tuples)
arr = pd.DataFrame(values,index=files.keys(), columns=cols)
arr.sort_values(by='arch',inplace=True)
arr.set_index(['arch','nodes'],inplace=True)
#arr.loc[:,[('dot','bw','avg'),('dot','lat','avg')]]
arr

axpby                      dot             ... dxdy4           \
                 bw     lat               bw     lat     ...   lat            
                avg std avg min corr std avg std avg min ...   avg min corr   
arch    nodes                                            ...                  
gtx1060 1       157   0  23   4    0   0  26   0 200 132 ...   323  72    0   
i5      1        30   0  31  12    0   1   9   0 317 117 ...  1212 208    0   
knl     1       393  22  12  10    5   0 141   7  77  63 ...   216  52    0   
        2       420  40  11  10    8   0 128   3  99  92 ...   208  92   60   
        4       421  44  11  10    9   0 110   4 126 122 ...   155  83   68   
p100    1       553   1   4   3    0   0 346   2  55  51 ...   148  30    0   
        2       554   0   3   3    2   0 340   3  53  50 ...   125  62   45   
        4       555   0   3   3    2   0 328   5  54  49 ...   101  70   61   
skl     1       207   6   5   4    0   0 192  18  32  24 ...   341  83   15   
        2       216   7   4   4    0   0 182  12  31  26 ...   149  64   31   
        4       233  15   4   4    0   0 167  24  44  42 ...    91  47   30   
v100    1       848   1   4   3    1   0 593   2  37  36 ...    47  12    2   
        2       847   4   3   3    2   0 580  11  39  38 ...    71  52   46   
        4       845   2   3   3    2   0 551  10  39  35 ...    78  71   67   

                  dxdy5                        
                     bw      lat               
              std   avg std  avg min corr std  
arch    nodes                                  
gtx1060 1       0    69  17  571 125    0   0  
i5      1      12    21   2 1957 340    0   1  
knl     1       2   101  15  452  98    0  38  
        2       2    88   7  363 134   71   9  
        4       5    89  14  236 108   77   6  
p100    1       0   165  14  273  63    3   0  
        2       0   159  11  188  75   45   0  
        4       1   150  15  134  74   58   1  
skl     1       1   111   8  443 133   20   2  
        2       1   114   7  250  89   35   1  
        4       1   110   9  143  62   34   3  
v100    1       0   535  49   85  24    5   1  
        2       1   463  75   93  58   48   1  
        4       2   408  83   89  72   66   2  

[14 rows x 36 columns]

In [77]:
#arr=arr.reset_index()

In [78]:
#define conversion function 
def toString(x): 
    if pd.isnull(x) : return 'n/a'
    #string = '%.1f'% x
    string = '%d' %np.ceil(x)
    #if np.ceil(x)<100 : string = '0'+string
    if np.ceil(x)<10 : string = '0'+string
    return string

In [84]:
addto = []
for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    arr.loc[:,(n,'bw','string')]= arr[n]['bw']['avg'].apply(toString) +" ± "+arr[n]['bw']['std'].apply(toString)
    arr.loc[:,(n,'lat','string')]= arr[n]['lat']['corr'].apply(toString) +" ± "+arr[n]['lat']['std'].apply(toString)
    addto.append((n,'lat','string'))
    addto.append((n,'bw','string'))

#make a table for display
nicetable=arr[addto]
drop = nicetable.columns.droplevel(2)
nicetable.columns=drop
#nicetable.reset_index(inplace=True)
#nicetable.set_index('arch')
newindex=[('i5',1)]
newindex.append(('gtx1060',1))
for n in ['skl','knl']:
    for m in [1,2,4]:
        newindex.append((n,m))
for n in ['p100','v100']:
    for m in [1,2,4]:
        newindex.append((n,m))
    
nicetable=nicetable.reindex(newindex)

nicetable

axpby                 dot              dxdy2            \
                   lat        bw       lat        bw      lat        bw   
arch    nodes                                                             
i5      1      00 ± 02   30 ± 01   05 ± 01   10 ± 01  00 ± 02   28 ± 03   
gtx1060 1      00 ± 01  158 ± 01   93 ± 09   27 ± 01  00 ± 01  131 ± 01   
skl     1      00 ± 01  207 ± 06   18 ± 03  193 ± 19  23 ± 03  182 ± 36   
        2      00 ± 01  217 ± 08   23 ± 01  183 ± 12  30 ± 03  175 ± 45   
        4      00 ± 01  233 ± 16   38 ± 05  167 ± 24  29 ± 03  168 ± 44   
knl     1      06 ± 01  394 ± 23   55 ± 02  142 ± 07  10 ± 01  240 ± 18   
        2      08 ± 01  421 ± 41   87 ± 02  129 ± 03  49 ± 01  176 ± 28   
        4      10 ± 01  421 ± 44  120 ± 06  110 ± 04  53 ± 04  156 ± 25   
p100    1      01 ± 01  554 ± 01   49 ± 01  347 ± 02  02 ± 01  288 ± 03   
        2      02 ± 01  555 ± 01   48 ± 02  340 ± 03  46 ± 01  249 ± 20   
        4      03 ± 01  555 ± 01   49 ± 01  329 ± 06  64 ± 01  228 ± 16   
v100    1      02 ± 01  849 ± 01   34 ± 02  594 ± 03  04 ± 01  802 ± 17   
        2      03 ± 01  847 ± 05   37 ± 02  581 ± 11  50 ± 01  575 ± 94   
        4      03 ± 01  845 ± 02   35 ± 01  552 ± 11  67 ± 02  469 ± 57   

                 dxdy3               dxdy4              dxdy5            
                   lat         bw      lat        bw      lat        bw  
arch    nodes                                                            
i5      1      00 ± 04    30 ± 03  00 ± 13   26 ± 02  00 ± 02   22 ± 02  
gtx1060 1      03 ± 01   112 ± 02  00 ± 01   84 ± 14  00 ± 01   70 ± 18  
skl     1      17 ± 01   162 ± 13  16 ± 02  119 ± 19  21 ± 03  111 ± 09  
        2      31 ± 03   158 ± 17  31 ± 01  121 ± 21  35 ± 01  115 ± 08  
        4      30 ± 02   160 ± 08  30 ± 02  115 ± 25  34 ± 03  111 ± 10  
knl     1      08 ± 02   173 ± 27  00 ± 02  127 ± 19  00 ± 38  102 ± 15  
        2      56 ± 03   142 ± 23  60 ± 02  110 ± 15  72 ± 09   89 ± 08  
        4      59 ± 04   128 ± 24  68 ± 06  116 ± 22  77 ± 07   90 ± 14  
p100    1      01 ± 01   238 ± 04  00 ± 01  201 ± 02  04 ± 01  166 ± 15  
        2      44 ± 01   217 ± 14  45 ± 01  186 ± 13  45 ± 01  160 ± 12  
        4      62 ± 01   193 ± 24  62 ± 02  175 ± 15  59 ± 01  151 ± 16  
v100    1      03 ± 01   713 ± 20  03 ± 01  650 ± 16  06 ± 01  536 ± 49  
        2      49 ± 01   576 ± 81  47 ± 01  536 ± 82  48 ± 01  463 ± 75  
        4      67 ± 02  438 ± 115  68 ± 02  468 ± 80  66 ± 02  408 ± 84

#### Assumptions
- there are three basic functions: trivially parallel(axpby), nearest neighbor (dxdy), global reduction (dot)
- each can be represented by the single node bandwidth, the single node latency and the multinode latency

#### But
- does not capture cache effect e.g. in SKl

In [80]:
index = ['i5','gtx1060','skl','knl','p100','v100']  
lines = []
for arch in  index: 
    line = []
    line.append(arch)
    #first the bandwidths
    for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
        line.append( arr.loc[(arch,1),(n,'bw','avg')] )
        line.append( arr.loc[(arch,1),(n,'bw','std')])
    for n in ['axpby','dot','dxdy2'] :
        line.append( arr.loc[(arch,1),(n,'lat','corr')] )
        line.append( arr.loc[(arch,1),(n,'lat','std')])
        if arch == 'i5' or arch == 'gtx1060':
            line.append(None)
            line.append(None)
        else:
            line.append( arr.loc[(arch,4),(n,'lat','corr')] )
            line.append( arr.loc[(arch,4),(n,'lat','std')] )
    lines.append(line)
    
tuples=['arch']     

for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append(n+'_bw')
    tuples.append(n+'_bw_err')
for n in ['axpby','dot','dxdy']:
    tuples.append(n+'_lat_shared')
    tuples.append(n+'_lat_shared_err')
    tuples.append(n+'_lat_dist')
    tuples.append(n+'_lat_dist_err')
cols=tuples
toDisk = pd.DataFrame(lines, columns=cols)
toDisk.to_csv('performance.csv',sep=' ',index=False)

In [81]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
test = pd.read_csv('performance.csv',delimiter=' ')
test

,arch,axpby_bw,axpby_bw_err,dot_bw,dot_bw_err,dxdy2_bw,dxdy2_bw_err,dxdy3_bw,dxdy3_bw_err,dxdy4_bw,...,axpby_lat_dist,axpby_lat_dist_err,dot_lat_shared,dot_lat_shared_err,dot_lat_dist,dot_lat_dist_err,dxdy_lat_shared,dxdy_lat_shared_err,dxdy_lat_dist,dxdy_lat_dist_err
0,i5,29.99,0.19,9.31,0.04,27.79,2.97,29.12,2.84,25.58,...,nan,nan,4.76,0.23,nan,nan,0.00,1.44,nan,nan
1,gtx1060,157.05,0.06,26.50,0.10,130.63,0.40,111.23,1.11,83.82,...,nan,nan,92.06,8.70,nan,nan,0.00,0.82,nan,nan
2,skl,206.71,5.87,192.05,18.31,181.56,35.38,161.75,13.00,118.06,...,0.00,0.26,17.28,2.32,37.93,4.14,22.70,2.11,28.52,2.10
3,knl,393.15,22.19,141.36,6.63,239.04,17.02,172.69,26.80,126.04,...,9.16,0.09,54.83,1.79,119.59,5.14,9.93,0.70,52.67,3.72
4,p100,553.33,0.69,346.08,1.53,287.57,2.14,237.35,3.26,200.30,...,2.42,0.05,48.15,0.71,48.16,0.63,1.82,0.34,63.51,0.80
5,v100,848.36,0.67,593.35,2.15,801.91,16.97,712.07,19.24,649.44,...,2.30,0.11,33.95,1.15,34.73,0.56,3.11,0.16,66.63,1.22


#### Observations
- note the high latency in the knl MPI implementation of dxdy. It seems to suffer from the same problem as the GPUs. (Is this the speed of PCIe we see?)

In [91]:
#index = ['i5','skl','knl','gtx1060','p100','v100']  
#theo = [38,None,None,192,732,898]
lines = []
for arch in  index: 
    line = []        
    for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
        bw  = arr.loc[(arch,1),(n,'bw','avg')]
        bw_err = arr.loc[(arch,1),(n,'bw','std')]
        lat1 = arr.loc[(arch,1),(n,'lat','corr')]
        lat1_err = arr.loc[(arch,1),(n,'lat','std')]
        line.append( toString( bw)+" $\pm$ "+toString(bw_err))
        line.append( toString( lat1)+" $\pm$ "+toString(lat1_err) )
        if arch == 'i5' or arch == 'gtx1060':
            line.append(toString(None))
        else:
            if (n == 'dot') or (n == 'axpby'):
                lat4 = arr.loc[(arch,4),(n,'lat','corr')]
                lat4_err = arr.loc[(arch,4),(n,'lat','std')]
            else:
                lat4 = arr.loc[(arch,4),('dxdy2','lat','corr')]
                lat4_err = arr.loc[(arch,4),('dxdy2','lat','std')]  
            line.append( toString( lat4)+" $\pm$ "+toString(lat4_err))

                
    lines.append(line)
    
tuples=[]  


for p in ['axpby','dot','B(P=2)','B(P=3)','B(P=4)','B(P=5)']:
    #for q in ['efficiency [\% bw]','lat s [us]','lat d [us]']:
    for q in ['bandwidth [GB/s]','$T_{lat}(1)$ [$\mu$s]','$T_{lat}(4)$ [$\mu$s]']:
        tuples.append((p,q))
tuples[0] = ('axpby','bandwidth [GB/s]')
    

cols=pd.MultiIndex.from_tuples(tuples)

toDisk = pd.DataFrame(lines, index=index, columns=cols)
#toDisk.insert(0,('theo','[GB/s]'),theo)
pd.set_option('display.float_format', lambda x: '%.0f' % x)
filename='axpby-dot.tex'
with open(filename, 'wb') as f:
    f.write(bytes(toDisk.iloc[:,0:6].to_latex(
        escape=False,
        column_format='lp{1.7cm}p{1.2cm}p{1.2cm}p{1.7cm}p{1.2cm}p{1.4cm}',
        bold_rows=True),'UTF-8'))
toDisk.iloc[:,0:6]

axpby                                              \
        bandwidth [GB/s] $T_{lat}(1)$ [$\mu$s] $T_{lat}(4)$ [$\mu$s]   
i5           30 $\pm$ 01           00 $\pm$ 02                   n/a   
gtx1060     158 $\pm$ 01           00 $\pm$ 01                   n/a   
skl         207 $\pm$ 06           00 $\pm$ 01           00 $\pm$ 01   
knl         394 $\pm$ 23           06 $\pm$ 01           10 $\pm$ 01   
p100        554 $\pm$ 01           01 $\pm$ 01           03 $\pm$ 01   
v100        849 $\pm$ 01           02 $\pm$ 01           03 $\pm$ 01   

                     dot                                              
        bandwidth [GB/s] $T_{lat}(1)$ [$\mu$s] $T_{lat}(4)$ [$\mu$s]  
i5           10 $\pm$ 01           05 $\pm$ 01                   n/a  
gtx1060      27 $\pm$ 01           93 $\pm$ 09                   n/a  
skl         193 $\pm$ 19           18 $\pm$ 03           38 $\pm$ 05  
knl         142 $\pm$ 07           55 $\pm$ 02          120 $\pm$ 06  
p100        347 $\pm$ 02           49 $\pm$ 01           49 $\pm$ 01  
v100        594 $\pm$ 03           34 $\pm$ 02           35 $\pm$ 01

In [92]:
dxdy = toDisk.loc[:,[('B(P=2)','bandwidth [GB/s]'),
                     ('B(P=3)','bandwidth [GB/s]'),
                     ('B(P=4)','bandwidth [GB/s]'),
                     ('B(P=5)','bandwidth [GB/s]'),
                     ('B(P=2)','$T_{lat}(1)$ [$\mu$s]'),
                     ('B(P=2)','$T_{lat}(4)$ [$\mu$s]'),
                    ]]

dxdy.columns.set_levels(['B(P=2) [GB/s]','B(P=3) [GB/s]','B(P=4) [GB/s]','B(P=5) [GB/s]',
                         '$T_{lat}(1)$ [$\mu$s]','$T_{lat}(4)$ [$\mu$s]'],level=1,inplace=True)
dxdy.columns.set_labels([0,1,2,3,4,5],level=1,inplace=True)
dxdy.columns=dxdy.columns.droplevel(0)
filename='dxdy.tex'
with open(filename, 'wb') as f:
    f.write(bytes(dxdy.to_latex(
        escape=False,column_format='lp{1.5cm}p{1.5cm}p{1.5cm}p{1.5cm}p{1.2cm}p{1.2cm}',
        bold_rows=True),'UTF-8'))
dxdy

,B(P=2) [GB/s],B(P=3) [GB/s],B(P=4) [GB/s],B(P=5) [GB/s],$T_{lat}(1)$ [$\mu$s],$T_{lat}(4)$ [$\mu$s]
i5,28 $\pm$ 03,30 $\pm$ 03,26 $\pm$ 02,22 $\pm$ 02,00 $\pm$ 02,n/a
gtx1060,131 $\pm$ 01,112 $\pm$ 02,84 $\pm$ 14,70 $\pm$ 18,00 $\pm$ 01,n/a
skl,182 $\pm$ 36,162 $\pm$ 13,119 $\pm$ 19,111 $\pm$ 09,23 $\pm$ 03,29 $\pm$ 03
knl,240 $\pm$ 18,173 $\pm$ 27,127 $\pm$ 19,102 $\pm$ 15,10 $\pm$ 01,53 $\pm$ 04
p100,288 $\pm$ 03,238 $\pm$ 04,201 $\pm$ 02,166 $\pm$ 15,02 $\pm$ 01,64 $\pm$ 01
v100,802 $\pm$ 17,713 $\pm$ 20,650 $\pm$ 16,536 $\pm$ 49,04 $\pm$ 01,67 $\pm$ 02
